In [ ]:
import os
import sys
import cv2
import pandas as pd
import numpy as np
sys.path.append('../../')

from utils.io import load_yaml


server = 'local'
server = load_yaml('../../config/servers.yml')[server]
prefix = server['database']['location']

def create_mask(img, center, radius):
    h, w, _ = img.shape

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[1])**2 + (Y-center[0])**2)

    mask = dist_from_center <= radius
    binary = np.zeros_like(mask, float)
    binary[mask] = 1
    # cv2.imshow('mask', binary)
    return binary

In [ ]:
import pandas as pd

from utils.io import recursive_scan2df, generate_path
from utils.sampling import RandomSequences
from utils import endoscopy

log_df = pd.DataFrame(columns=['folder', 'frame', 'segmentation'])

offset = 5

cnt = 1062
frame_prefix = 'out/frame'
segme_prefix = 'out/segmentation'
generate_path(frame_prefix)
generate_path(segme_prefix)

ris_path = os.path.join(prefix, 'ROBUST_MIS_raw_data')
ris = recursive_scan2df(ris_path, '.avi')
ris.folder = ris.folder.apply(lambda x: os.path.join(ris_path, x))
cho_path = os.path.join(prefix, 'cholec80/videos')
cho = recursive_scan2df(cho_path, '.mp4')
cho.folder = cho.folder.apply(lambda x: os.path.join(cho_path, x))

# df = pd.concat([ris, cho])
df = cho

paths = [os.path.join(row.folder, row.file) for _, row in df.iterrows()]

rs = RandomSequences(
    max_seq=3000,
    paths=paths,
    seq_len=1
)

th = 0.01
min = 10
max = 0.2
steps = 200

for frames in rs:
    cv2.namedWindow('circle_frame')
    cv2.moveWindow('circle_frame', 0, 0)
    for frame in frames:
        circle_frame = frame[offset:-offset, offset:-offset].copy()

        mask = endoscopy.bilateralSegmentation(circle_frame, th=th)
        binary = np.zeros_like(frame[:,:,0].squeeze())
        center, radius = endoscopy.ransacBoundaryCircle(mask, decay=1., n_pts=10, fit='numeric', n_iter=1)


        def on_change(value):
            circle_frame = frame[offset:-offset, offset:-offset].copy()
            global center, radius, binary
            mask = endoscopy.bilateralSegmentation(circle_frame, th=value/steps*max)
            center, radius = endoscopy.ransacBoundaryCircle(mask, decay=1., n_pts=10, fit='numeric', n_iter=1)
            center, radius = center.astype(int), int(radius)
            
            if radius is not None:
                cv2.circle(circle_frame, (center[1], center[0]), radius, (255, 255, 0), 1)
                cv2.imshow('circle_frame', circle_frame)
                binary = create_mask(frame, center + offset, radius)
                # cv2.imshow('mask', mask)

        def keep_change(value):
            # append df
            global center, radius, log_df, cnt

            log_df = log_df.append(
                {
                    'frame': 'frame/{:04d}.npy'.format(cnt),
                    'segmentation': 'segmentation/{:04d}.npy'.format(cnt)
                },
                ignore_index=True
            )
            np.save('out/frame/{:04d}.npy'.format(cnt), frame)
            np.save('out/segmentation/{:04d}.npy'.format(cnt), binary)
            cnt += 1                

        if radius is not None:
            center, radius = center.astype(int), int(radius)

            cv2.circle(circle_frame, (center[1], center[0]), radius, (255, 255, 0), 1)

            cv2.imshow('circle_frame', circle_frame)
            # cv2.imshow('mask', mask)
            cv2.createTrackbar('th', 'circle_frame', min, steps, on_change)
            cv2.createTrackbar('keep', 'circle_frame', 0, 1, keep_change)
            cv2.waitKey()
            cv2.destroyAllWindows()

In [ ]:
import numpy as np
import sys
sys.path.append('../..')
import cv2

from utils.io import recursive_scan2df


prefix = '/media/martin/Samsung_T5/data/endoscopic_data/boundary_segmentation'
df = recursive_scan2df(os.path.join(prefix, 'frame'), '.npy')

for _, row in df.iterrows():
    file = row.file
    img = np.load(os.path.join(prefix, 'frame', file))
    seg = np.load(os.path.join(prefix, 'segmentation', file))

    # cv2.imshow('img', np.multiply(img, np.expand_dims(seg, -1)/255.))
    cv2.imshow('img', img)
    cv2.imshow('seg', seg)
    cv2.waitKey()
cv2.destroyAllWindows()